In [22]:
from collections import deque


class Automato:
    """Implementa um automato finito determinístico (AFD) que obedece a 
    seguinte definição:
    M = (Q, Σ, δ, q0, F)
    Q: Conjunto finito de estados
    Σ: Alfabeto
    δ: Função de transição
    q0: Estado inicial
    F: Conjunto de estados finai
    """
    
    estados_finitos: set[str]
    alfabeto: set[str]
    transicoes: dict[dict[str, str]]
    estado_inicial: str
    conj_estados_finais: set[str]
    fita: deque[str]

    def __init__(self, estados_finitos, alfabeto, transicoes,
                estado_inicial, conj_estados_finais):
        self.estados_finitos: set = estados_finitos
        self.alfabeto: set = alfabeto
        self.transicoes = transicoes
        self.estado_inicial = estado_inicial
        self.estados_finais = conj_estados_finais
        self.fita = deque()
        self.cursor = 0
        self.maquina_estados: MaquinaEstados = MaquinaEstados(self)

    def __repr__(self) -> str:
        return f"""\
        M = (Q, Σ, δ, q0, F)
        Q: {{{','.join(self.estados_finitos)}}}
        Σ: {{{self.alfabeto}}}
        δ: {self.maquina_estados.transicao}
        q0: {self.estado_inicial}
        F: {self.estados_finais}\
        """
    
    def __eq__(self, o: "Automato") -> bool:
        """Retorna True se os dois automatos forem equivalentes."""
        # Aqui é preciso aplicar a técnica de minimização em ambos 
        # automatos, o que implica implementar 1. os algoritmos de remoção
        # de transições em vazio; 2. de remoção de não-determinismos;
        # 3. de remoção de estados inúteis ou inacessíveis, e o
        # algoritmo de minimização, nesta ordem.
        #
        # minimo_self = self.rm_transicoes_vazio().rm_nao_determinismo().rm_estados_inuteis()
        # minimo_o = o.rm_transicoes_vazio().rm_nao_determinismo().rm_estados_inuteis()
        pass

    def escrever_fita(self, cadeia: str) -> None:
        """Escreve uma cadeia de caracteres na fita."""
        self.fita.clear()
        self.fita = deque(cadeia)
    
    def ler_celula(self) -> str:
        """Retorna o conteudo da celula atual da fita."""
        self.cursor += 1
        return self.fita.pop()

    def reconhecer_cadeia_alt(self, cadeia: str):
        self.maquina_estados.resetar()    
        self.escrever_fita(cadeia)
        self.maquina_estados.gravar_movimento(self.fita)
        while self.fita:
            simbolo = self.fita.popleft()
            try:
                self.maquina_estados.config_seguinte(simbolo)
                self.maquina_estados.gravar_movimento(self.fita)
            except KeyError:
                return False
        estado_parada = self.maquina_estados.estado_atual
        return bool(estado_parada in self.estados_finais)
    
    def rm_transicoes_vazio(self) -> "Automato":
        """Retorna um novo automato sem transições em vazio."""
        # TODO
        pass

    def rm_nao_determinismo(self) -> "Automato":
        """Retorna um novo automato sem não-determinismos."""
        # TODO
        pass

    def rm_estados_inuteis(self) -> "Automato":
        """Retorna um novo automato sem estados inuteis."""
        # TODO
        pass


# Maquina de estados
class MaquinaEstados:
    """Classe que implementa uma máquina de estados finitos 
    determinística por composição da classe Automato.
    """
    
    funcao_transicao: dict[dict[str, str]]
    estado_inicial: str
    estados_finais: set[str]


    def __init__(self, automato: Automato):
        self.transicao = automato.transicoes
        self.estados_finais = automato.estados_finais
        self.estado_inical = automato.estado_inicial
        self.estado_atual = automato.estado_inicial
        self.cadeia_restante = None
        self.posicao_cursor = 0
        self.movimentos = list()
    
    def config_seguinte(self, simbolo):
        """Aplica a função de transição δ(p, σ) -> q."""
        if simbolo != "\0":
            self.estado_atual = self.transicao[self.estado_atual][simbolo]
            self.posicao_cursor += 1

    def gravar_movimento(self, cadeia: deque[str]):
        """Retorna o histórico de movimentos da máquina de estados."""
        self.movimentos.append((self.estado_atual, "".join(cadeia)))

    def apresentar_movimentos(self):
        mov_formatados = str(self.movimentos).replace("), ", ") ⊢ ")
        return mov_formatados

    def resetar(self):
        """Reseta a máquina de estados."""
        self.estado_atual = self.estado_inical
        self.cadeia_restante = None
        self.posicao_cursor = 0
        self.movimentos = list()

    # Opcao com recursao
    def reconhecer_cadeia(self, cadeia):
        """Reconhece uma cadeia inteira na fita, de acordo com a 
        função de transição.
        """

        if self.cadeia_restante == None:
            self.movimentos = []
            self.movimentos.append((self.estado_atual, cadeia))
        
        resultado = False
        if (self.estado_atual, cadeia[0]) in self.transicao.keys():
        
            self.estado_atual = self.transicao[(self.estado_atual, cadeia[0])]
            self.cadeia_restante = cadeia[1:]
            self.movimentos.append((self.estado_atual, self.cadeia_restante))
            if self.cadeia_restante == "":
                return True if self.estado_atual in self.estados_finais else False
            else:
                resultado = self.reconhecer_cadeia(self.cadeia_restante)
        else:            
            return False
        self.estado_atual = self.estado_inical
        self.cadeia_restante = None
        return resultado


In [23]:
estados = set(["q0", "q1", "q2", "q3"])
alfabeto = set(['0', '1', '2'])
funcao_de_transicao = {
    ('q0', '0'): 'q0',
    ('q0', '1'): 'q1',
    ('q0', '2'): 'q3',
    ('q1', '0'): 'q3',
    ('q1', '1'): 'q1',
    ('q1', '2'): 'q2',
    ('q2', '0'): 'q3',
    ('q2', '1'): 'q3',
    ('q2', '2'): 'q2',
    ('q3', '0'): 'q3',
    ('q3', '1'): 'q3',
    ('q3', '2'): 'q3',
}
estado_inicial = 'q0'
estados_finais = set(['q1', 'q2'])

In [24]:
automato_1 = Automato(estados, alfabeto, funcao_de_transicao, estado_inicial, estados_finais)

In [28]:
print(automato_1.maquina_estados.reconhecer_cadeia('0001112'))

True


In [29]:
automato_1.escrever_fita('0001112')
print(automato_1.fita)

deque(['0', '0', '0', '1', '1', '1', '2'])


In [30]:
from collections import deque

def escrever_fita(fita: deque, cadeia: str) -> None:
    """Escreve uma cadeia de caracteres na fita."""
    fita.clear()
    fita = deque(cadeia)
    fita.append("\0")
    return fita

fita = deque()
fita = escrever_fita(fita, "0001112")
fita.append("\0")
print(fita)

deque(['0', '0', '0', '1', '1', '1', '2', '\x00', '\x00'])


In [31]:
funcao_de_transicao2 = {
    'q0': {
        '0': 'q0',
        '1': 'q1',
        '2': 'q3',
    },
    'q1': {
        '0': 'q3',
        '1': 'q1',
        '2': 'q2',
    },
    'q2': {
        '0': 'q3',
        '1': 'q3',
        '2': 'q2',
    },
    'q3': {
        '0': 'q3',
        '1': 'q3',
        '2': 'q3',
    },    
}

In [32]:
from itertools import accumulate
from collections import deque
from functools import reduce

def muda_estado(estado_atual, simbolo) -> str:
    estado_atual = funcao_de_transicao2[estado_atual][simbolo]
    return estado_atual

estado_atual = "q0"
estado_atual = muda_estado(estado_atual, "2")
print(estado_atual)

q3


In [33]:
cadeia = deque("0001112")
transicoes = accumulate(cadeia, muda_estado, initial='q0')
transicoes2 = reduce(muda_estado, cadeia, 'q0')

In [36]:
print(automato_1.maquina_estados.reconhecer_cadeia("0011222"))
print(automato_1.maquina_estados.movimentos)
print(str(automato_1.maquina_estados.movimentos).replace("), ", ") ⊢ "))

True
[('q0', '0011222'), ('q0', '011222'), ('q0', '11222'), ('q1', '1222'), ('q1', '222'), ('q2', '22'), ('q2', '2'), ('q2', '')]
[('q0', '0011222') ⊢ ('q0', '011222') ⊢ ('q0', '11222') ⊢ ('q1', '1222') ⊢ ('q1', '222') ⊢ ('q2', '22') ⊢ ('q2', '2') ⊢ ('q2', '')]


#### Conversão de RegEx em AFND
O autômato finito não-determinístico que aceita a linguagem definida por r pode ser obtido através da aplicação do algoritmo a seguir que especifica as regras de mapeamento parciais que abrangem casos triviais de sentenças (itens 1, 2 e 3) e cada um dos operadores de união (4), concatenação (5) e fechamento (6)

Obs:. F são finais de segmento
```
Casos triviais:
1. ->   q0: {
        ε: q1
        }
        Q = {q0, q1}
        F = {q1}

2. ->   q0: {
        NULL:q0
        }
        Q = {q0, q1}
        F = {q1}

3. ->   q0: {
        σ: q1
        }
        Q = {q0, q1}
        F = {q1}

Operadores:
4. a|b = {
    q0: {
        ε: q1,
        ε: q2,
    },
    q1: {
        a: q2
    },
    q3: {
        b: q4
    },
    q4: {
        ε: q5
    },
    q2: {
        ε: q5
    }
}
    Q = {q0, q1, q2, q3, q4, q5}
    F = {q5}

5. ab = {
    q0: {
        a: q1
    },
    q1: {
        ε: q2
    },
    q2: {
        b: q3
    },
    q3: {
        ε: q4
    }
}
    Q = {q0, q1, q2, q3, q4}
    F = {q4}

6. a* = {
    q0: {
       ε: q1,
       ε: q5, 
    }
    q1: {
        ε: q2
    },
    q2: {
        ε: q3
    },
    q3: {
        ε: q4
    },
    q4: {
        ε: q5,
        ε: q1
    }
}
    Q = {q0, q1, q2, q3, q4, q5}
    F = {q5}
```

passo-a-passo: 
1. encontrar os parenteses e resolver as expressões.
2. aplicar a funcao a cada membro da expressão regular.


In [37]:
# expressao = "ab*"

# for simbolo in regex:
#     if simbolo == "(":
#         regex.index(simbolo)
#         regex.index(")")
#         pass



#### Implementação usando namedtuples (mais barata)

In [38]:
from collections import namedtuple
import pandas as pd


def construir_transicoes(alfabeto: str, epsilon=False) -> namedtuple:
    """Constrói uma namedtuple com o alfabeto da linguagem e se o
    automato aceita transições vazias.
        Args:
            alfabeto (str) -- Alfabeto da linguagem.
            epsilon (bool, optional) -- Se o automato aceita transições 
                                        vazias. Defaults to False.
        Returns:
            namedtuple: Tupla com o alfabeto e se aceita transições vazias.
    """
    fields = f'{" ".join(alfabeto) + (" epsilon" if epsilon else "")}'
    alfabeto_tuple = namedtuple('transita', fields)
    return alfabeto_tuple

def construir_bloco_or(a: str, b: str, n: int) -> namedtuple:
    """Constrói um bloco de transições para o operador OR de acordo com
    o algoritmo de Thompson.
        Args:
            a (str) -- Primeiro operando.
            b (str) -- Segundo operando.
            n (int) -- Número do bloco.
        Returns:
            namedtuple: Tupla com os estados do bloco.
    """
    transicoes = construir_transicoes(f"{a}{b}", epsilon=True)
    bloco_or = namedtuple('estado', f'q{n}0 q{n}1 q{n}2 q{n}3 q{n}4')

    q0 = transicoes('','',(f'q{n}1', f'q{n}2'))
    q1 = transicoes(f'q{n}2','','')
    q2 = transicoes('','',f'q{n}5')
    q3 = transicoes('',f'q{n}4','')
    q4 = transicoes('','',f'q{n}5')

    return bloco_or(q0, q1, q2, q3, q4)

def construir_fecho(simbolo: str) -> namedtuple:
    """Constrói um bloco de transições para o operador de fecho de Kleene
    de acordo com o algoritmo de Thompson.
        Args:
            simbolo (str) -- operando do operador de fecho de Kleene.
        Returns:
            namedtuple: Tupla com os estados do bloco.
    """
    pass

bloco_teste_2 = construir_bloco_or('c', 'd', 0)
bloco_teste_2_df = pd.DataFrame(bloco_teste_2, index=bloco_teste_2._fields)
bloco_teste_2_df

,c,d,epsilon
q00,,,"(q01, q02)"
q01,q02,,
q02,,,q05
q03,,q04,
q04,,,q05


#### Implementação usando dicionário (mais cara)

In [39]:
bloco_or_dict = {
    'q00': {
        'c': '',
        'd': '',
        'epsilon': ('q01', 'q02')
        },
    'q01': {
        'c': 'q02',
        'd': '',
        'epsilon': ''
        },
    'q02': {
        'c': '',
        'd': '',
        'epsilon': 'q05'
        },
    'q03': {
        'c': '',
        'd': 'q04',
        'epsilon': ''
        },
    'q04': {
        'c': '',
        'd': '',
        'epsilon': 'q05'}
    }
bloco_teste_3_df = pd.DataFrame(bloco_or_dict).T
bloco_teste_3_df.loc['q00'].epsilon

('q01', 'q02')

In [40]:
nao_determinismo = any(isinstance(simbolo, tuple) for estado in bloco_teste_2\
                    for simbolo in estado)
print(nao_determinismo)

True


In [41]:
class Estado:
    """Classe que representa um estado de um autômato finito."""
    
    func_transicao: dict[str, dict[str, 'Estado']]
    func_transdutora: str
    nome: str

    def __init__(self, nome, func_transicao={}, func_transdutora=""):
        self.nome = nome
        self.func_transicao = func_transicao
        self.func_transdutora = func_transdutora

    def __str__(self):
        descricao = f"{self.nome}:\
                    \nTrancicoes: {self.func_transicao}\
                    \nCadeia de saida: {self.func_transdutora}\n"
        return descricao

    def __repr__(self):
        return self.nome

    def set_func_transicao(self, func_transicao) -> None:
        self.func_transicao = func_transicao

    def set_func_transdutora(self, func_transdutora) -> None:
        self.func_transdutora = func_transdutora

q0 = Estado("q0",func_transdutora='0')
q0.set_func_transicao({'0': q0, '1': q0})

q1 = Estado("q1",func_transdutora='1')
q1.set_func_transicao({'0': q1, '1': q1})

q2 = Estado("q2",func_transdutora='')
q2.set_func_transicao({'0': q2, '1': q2})

print(q1, q2, q0)

q1:                    
Trancicoes: {'0': q1, '1': q1}                    
Cadeia de saida: 1
 q2:                    
Trancicoes: {'0': q2, '1': q2}                    
Cadeia de saida: 
 q0:                    
Trancicoes: {'0': q0, '1': q0}                    
Cadeia de saida: 0



In [42]:
class Transdutor(Automato):
    pass
        
automato_2 = Transdutor(estados, alfabeto, funcao_de_transicao2, estado_inicial, estados_finais)

In [43]:
automato_2.estados_finitos
print(automato_2.maquina_estados.reconhecer_cadeia('00000'))
print(isinstance(automato_2, Automato))

False
True


In [44]:
# Breadth First Search (BFS)
# DEVE SER UTIL PARA OS AFN-E

grafo = automato_2.transicoes
print(grafo)
visitados = []
print(visitados)
def bfs(automato: Automato, inicio: str, fim: str = None):
    fila = deque([inicio])
    while fila and fim not in visitados:
        vertice = fila.popleft()
        if vertice not in visitados:
            visitados.append(vertice)
            automato.maquina_estados.estado_atual = vertice
            for vizinho in grafo[vertice].values():
                print(vizinho, vizinho not in visitados)
                if vizinho not in visitados:
                    fila.append(vizinho)
                    print(fila)
    return visitados

bfs(automato_2, 'q0', 'q3')

{'q0': {'0': 'q0', '1': 'q1', '2': 'q3'}, 'q1': {'0': 'q3', '1': 'q1', '2': 'q2'}, 'q2': {'0': 'q3', '1': 'q3', '2': 'q2'}, 'q3': {'0': 'q3', '1': 'q3', '2': 'q3'}}
[]
q0 False
q1 True
deque(['q1'])
q3 True
deque(['q1', 'q3'])
q3 True
deque(['q3', 'q3'])
q1 False
q2 True
deque(['q3', 'q3', 'q2'])
q3 False
q3 False
q3 False


['q0', 'q1', 'q3']

In [46]:
# # Breadth First Search (BFS)

# automato_2.maquina_estados.estado_atual = automato_2.estado_inicial
# # print(consumidos)
# def bfs(automato: Automato, cadeia: str):
#     grafo = automato.transicoes
#     print(grafo)
#     consumidos = deque([])
#     transitados = deque([])
#     automato.escrever_fita(cadeia)
#     while automato.fita and cadeia not in consumidos:
#         alvo = automato.fita.popleft()
#         vertice = automato.maquina_estados.estado_atual
#         if cadeia != "".join(consumidos) and alvo in grafo[vertice].keys():
#             consumidos.append(alvo)
#             print(consumidos)
#             transitados.append(vertice)
#             print(transitados)
#             automato.maquina_estados.estado_atual = grafo[vertice][alvo]
#     else:
#             consumidos.append(alvo)
#             print(consumidos)
#             transitados.append(vertice)
#             print(transitados)
#     print(transitados)
#     return consumidos

# print(bfs(automato_2, ''))

In [48]:
# def reconhecer_cadeia(self, cadeia: str):
#     self.maquina_estados.resetar()    
#     self.escrever_fita(cadeia)
#     self.maquina_estados.gravar_movimento(self.fita)
#     while self.fita:
#         simbolo = self.fita.popleft()
#         try:
#             self.maquina_estados.config_seguinte(simbolo)
#             self.maquina_estados.gravar_movimento(self.fita)
#         except KeyError:
#             return False
#     estado_parada = self.maquina_estados.estado_atual
#     return bool(estado_parada in self.estados_finais)

# reconhecer_cadeia(self_2, '011')
# print(self_2.maquina_estados.movimentos)
# print(self_2.maquina_estados.apresentar_movimentos())

In [54]:
automato_2.reconhecer_cadeia_alt('013')

False

In [50]:
def reconhecer_cadeia(automato: Automato, cadeia: str):
    automato.maquina_estados.resetar()
    automato.escrever_fita(cadeia)    
    ler, fita = automato.maquina_estados.config_seguinte, automato.fita
    for _ in (ler(celula) for celula in fita):        
        automato.maquina_estados.movimentos.append(automato.maquina_estados.estado_atual)
    estado_parada = automato.maquina_estados.estado_atual
    return bool(estado_parada in automato.estados_finais)

reconhecer_cadeia(automato_2, '011')
print(list(automato_2.maquina_estados.hist_movimentos()))

AttributeError: 'MaquinaEstados' object has no attribute 'hist_movimentos'

In [ ]:
grafo['q0'].keys()

dict_keys(['0', '1', '2'])